# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context.

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](https://github.com/kavgan/OpinRank/blob/master/OpinRankDatasetWithJudgments.zip) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference.

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [3]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html).



In [5]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""

    logging.info("reading file {0}...this may take a while".format(input_file))

    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f):

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")

2025-07-15 12:28:33,104 : INFO : reading file reviews_data.txt.gz...this may take a while
2025-07-15 12:28:33,105 : INFO : read 0 reviews
2025-07-15 12:28:33,873 : INFO : read 10000 reviews
2025-07-15 12:28:34,652 : INFO : read 20000 reviews
2025-07-15 12:28:35,543 : INFO : read 30000 reviews
2025-07-15 12:28:36,427 : INFO : read 40000 reviews
2025-07-15 12:28:37,344 : INFO : read 50000 reviews
2025-07-15 12:28:38,246 : INFO : read 60000 reviews
2025-07-15 12:28:39,031 : INFO : read 70000 reviews
2025-07-15 12:28:39,763 : INFO : read 80000 reviews
2025-07-15 12:28:40,719 : INFO : read 90000 reviews
2025-07-15 12:28:41,424 : INFO : read 100000 reviews
2025-07-15 12:28:42,120 : INFO : read 110000 reviews
2025-07-15 12:28:42,824 : INFO : read 120000 reviews
2025-07-15 12:28:43,547 : INFO : read 130000 reviews
2025-07-15 12:28:44,320 : INFO : read 140000 reviews
2025-07-15 12:28:45,067 : INFO : read 150000 reviews
2025-07-15 12:28:45,810 : INFO : read 160000 reviews
2025-07-15 12:28:46,542

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn.

In [9]:
#### TO-DO: create mode and run
from gensim.models import Word2Vec
# create the model
model = Word2Vec(documents, vector_size=100, window=5, min_count=1, workers=10,epochs=10)

2025-07-15 12:33:37,079 : INFO : collecting all words and their counts
2025-07-15 12:33:37,079 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-07-15 12:33:37,187 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2025-07-15 12:33:37,294 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2025-07-15 12:33:37,418 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2025-07-15 12:33:37,534 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2025-07-15 12:33:37,658 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2025-07-15 12:33:37,776 : INFO : PROGRESS: at sentence #60000, processed 11013727 words, keeping 76787 word types
2025-07-15 12:33:37,876 : INFO : PROGRESS: at sentence #70000, processed 12637529 words, keeping 83200 word types
2025-07-15 12:33:37,968 : INFO : PROG

## Now, let's look at some output
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words.

In [10]:
#### To do fit model to calculate similarity
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

2025-07-15 12:34:48,415 : WARNING : Effective 'alpha' higher than previous training cycles
2025-07-15 12:34:48,415 : INFO : Word2Vec lifecycle event {'msg': 'training model with 10 workers on 150060 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2025-07-15T12:34:48.415641', 'gensim': '4.3.3', 'python': '3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'train'}
2025-07-15 12:34:49,420 : INFO : EPOCH 0 - PROGRESS: at 14.94% examples, 4965642 words/s, in_qsize 19, out_qsize 0
2025-07-15 12:34:50,422 : INFO : EPOCH 0 - PROGRESS: at 31.06% examples, 5137640 words/s, in_qsize 18, out_qsize 1
2025-07-15 12:34:51,423 : INFO : EPOCH 0 - PROGRESS: at 50.53% examples, 5260447 words/s, in_qsize 19, out_qsize 0
2025-07-15 12:34:52,424 : INFO : EPOCH 0 - PROGRESS: at 69.43% examples, 5327534 words/s, in_qsize 18, out_qsize 1
2025-07-15 12:34:53,432 : I

(304387075, 415193590)

In [15]:
w1 = ["dirty"]
model.wv.most_similar (positive=w1,topn=6)

[('filthy', 0.8952178955078125),
 ('unclean', 0.8178995847702026),
 ('grubby', 0.8115321397781372),
 ('dusty', 0.8109852075576782),
 ('smelly', 0.7986074090003967),
 ('dingy', 0.7971614003181458)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`.

In [11]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9516838192939758),
 ('friendly', 0.8838683366775513),
 ('cordial', 0.8689355254173279),
 ('curteous', 0.8611297011375427),
 ('professional', 0.8593620657920837),
 ('attentive', 0.8490883111953735)]

In [12]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('germany', 0.7659302949905396),
 ('philadelphia', 0.7196246385574341),
 ('spain', 0.717460572719574),
 ('hawaii', 0.716503918170929),
 ('toronto', 0.6904525756835938),
 ('mexico', 0.6874945759773254)]

In [13]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('amazed', 0.8365273475646973),
 ('astonished', 0.8362224102020264),
 ('appalled', 0.8351449370384216),
 ('dismayed', 0.8157597780227661),
 ('horrified', 0.8125491738319397),
 ('stunned', 0.7971389889717102)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [16]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('mattress', 0.734644889831543),
 ('blanket', 0.7209509015083313),
 ('matress', 0.7164228558540344),
 ('duvet', 0.7086477875709534),
 ('quilt', 0.6860620975494385),
 ('pillowcase', 0.6710792183876038),
 ('sheets', 0.6612488627433777),
 ('foam', 0.6541821360588074),
 ('pillows', 0.6267200708389282),
 ('pads', 0.6222143769264221)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary.

In [17]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.7986074

In [18]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

0.99999994

In [19]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.34906778

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [20]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [21]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me.

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window.

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary.

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work.
